<a href="https://colab.research.google.com/github/yyinsomnia/build-cngpt/blob/main/play_gpt_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# init 环境和数据




In [1]:
# ! pip install transformers

# play embd

1. 首先还是要能learn的，不然就只有forward没有backward了

In [2]:
import torch

# 模拟一个batch的input
vocab_size = 3758  # 3500个常用字 + 256个ASCII + 2个[UNK] [MASK]
B, T = 8, 16

shape = (8, 16, 1) # (B, T, id)
token_ids = torch.randint(low=0, high=3758, size=shape)
print(token_ids)

tensor([[[ 951],
         [2189],
         [2155],
         [ 105],
         [ 447],
         [3740],
         [2683],
         [2492],
         [2708],
         [3524],
         [2501],
         [1120],
         [1576],
         [ 725],
         [ 875],
         [2654]],

        [[ 181],
         [3262],
         [ 888],
         [2741],
         [2820],
         [ 608],
         [2665],
         [ 732],
         [2132],
         [1529],
         [ 705],
         [3753],
         [ 512],
         [3737],
         [2071],
         [2309]],

        [[2918],
         [1098],
         [2776],
         [3629],
         [3266],
         [1204],
         [1654],
         [2193],
         [2455],
         [ 154],
         [3024],
         [1629],
         [1227],
         [1984],
         [3713],
         [2866]],

        [[2097],
         [1435],
         [ 462],
         [2670],
         [ 188],
         [ 897],
         [ 796],
         [3326],
         [3010],
         [ 158],
        

* 注意：这里的共3个维度，最后一维度也是向量。

In [3]:
# 第2个batch的第4个token，也是一个向量
print(token_ids[1, 3])

tensor([2741])


In [4]:
from torch import nn

embd_dim = 256

embd = nn.Embedding(vocab_size, embd_dim)

x = embd(token_ids)

print(x.shape) # (B, T, C) C就是embd_dim

torch.Size([8, 16, 1, 256])


发现shape和预期的不符合

pytorch的docs里面
Shape Input: (\*) Output: (\*, H)


In [5]:
x

tensor([[[[ 7.9044e-01, -4.6274e-01, -6.6431e-01,  ..., -7.5047e-01,
            1.1894e+00, -1.1299e+00]],

         [[ 6.7120e-01, -6.4540e-01, -1.5818e-01,  ..., -4.1572e-02,
            5.4884e-01, -4.7011e-01]],

         [[ 8.8286e-01, -2.4676e-01, -5.8604e-01,  ...,  2.7456e-01,
           -4.9549e-01,  1.7087e+00]],

         ...,

         [[ 3.7835e-01,  6.4716e-01,  1.0535e+00,  ..., -1.5698e-01,
            2.5416e-02, -6.5913e-01]],

         [[-5.6564e-01,  6.6039e-02, -2.6857e-01,  ...,  7.3935e-01,
            5.1131e-01, -1.9955e-01]],

         [[ 1.7279e-01, -1.6990e+00,  2.4947e+00,  ...,  2.1290e-01,
           -8.6340e-02, -8.0215e-01]]],


        [[[ 1.5225e+00,  1.5168e+00,  6.8566e-01,  ..., -4.3976e-01,
            9.6012e-01, -5.8410e-01]],

         [[ 3.2221e-01, -9.1751e-01,  1.0769e+00,  ..., -4.3456e-01,
            4.7245e-01,  8.6417e-01]],

         [[ 2.6101e-01,  4.5095e-01, -7.1495e-01,  ...,  5.3278e-01,
           -2.3575e-01,  8.5281e-01]],

  

仔细想一下是合理的。如果是图片，那有RGB 3个值，就变成了(B, H, W, 3)
这样RGB每个过完embd，应该是变成了(B, H, W, 3, embd_dim)

In [6]:
# 简单一点把最后一维弄成标量看看
shape = (8, 16,)
token_ids = torch.randint(low=0, high=vocab_size, size=shape)
print(token_ids)
# 这个输出是熟悉的感觉

tensor([[2820, 2344, 2173, 1719,  560, 1713, 2056, 1003, 3373, 3288, 1994, 1692,
         1785, 3689, 3426, 2652],
        [ 369,  974, 1903, 1365, 1346,  761, 3086, 1609,  650,  748,  788, 3150,
          875, 2635, 2943,  436],
        [3326, 1979, 3693,  739, 2944, 2652, 2030, 1900, 2071,  231, 2280, 1980,
         2682, 3041, 1047, 3103],
        [2846, 2745, 1294, 3678,  165,  397, 1764, 1575, 1586, 2438,  571, 2705,
         3175, 3184, 2128, 1847],
        [2943, 1853, 1375, 1507, 1652, 1078, 3545, 2400, 3527, 3189, 2484, 2200,
         1069,  620, 2052,  670],
        [1720,  937,  700,  354, 3105,  742, 3178, 1433, 3352, 2261, 3116,   61,
          702,  268,  982, 1666],
        [1573, 1763, 3232,   26, 2098, 1401, 2488,  789, 2925,  980, 3655,  334,
         1363,  821, 3592, 1311],
        [ 913, 2662, 3564, 2456, 2317, 2730, 2772, 1221, 2439, 1836, 1254, 1778,
         3112, 2447, 3209, 3121]])


In [7]:
x = embd(token_ids)

print(x.shape)

# 这次感觉对了

torch.Size([8, 16, 256])


In [8]:
x
# 观察了一下，这个值还是有点技巧的，大部分都在 +- 1.0 左右

tensor([[[ 1.6951e+00, -2.9907e-01,  5.5990e-01,  ..., -2.1972e-02,
           2.1995e-01,  9.7566e-01],
         [-1.3859e+00,  1.1260e+00, -9.1951e-01,  ..., -2.7017e-01,
          -3.6381e-01, -9.3985e-01],
         [ 1.6992e+00,  9.3352e-01, -2.2224e+00,  ..., -9.2359e-01,
          -1.7415e+00, -5.1062e-01],
         ...,
         [ 3.9704e-02,  3.0490e-02, -1.8918e-01,  ..., -7.6731e-02,
           1.2411e+00,  1.0503e+00],
         [ 6.5356e-01, -1.3521e-01,  1.3906e-01,  ..., -1.2789e+00,
          -1.0857e+00,  8.2030e-02],
         [-6.2034e-01,  8.1095e-01, -9.5309e-01,  ..., -1.2244e+00,
          -6.7006e-04,  6.0653e-01]],

        [[-2.2803e-01,  4.8385e-01, -1.3307e+00,  ..., -1.1325e-01,
          -7.1718e-01,  6.0544e-01],
         [ 5.7860e-01,  5.9058e-01, -8.3402e-01,  ..., -2.9735e-01,
           1.9023e+00, -9.1666e-01],
         [ 2.1054e+00, -1.1658e-01,  8.0937e-01,  ..., -3.7553e-01,
           5.6007e-01, -1.6357e+00],
         ...,
         [ 7.3999e-01,  1

# play attention
关键词：**QKV** , **wei matrix**, **tril**, **softmax**

对matrix的理解还有不够深刻，需要组织一段语言或者一个动图

In [9]:
headers_num = 8
header_dim = embd_dim // headers_num

query = nn.Linear(embd_dim, header_dim)
key = nn.Linear(embd_dim, header_dim)
value = nn.Linear(embd_dim, header_dim)

q = query(x) # shape(B, T, header_dim) = (8, 16, 32)
k = key(x)
v = value(x)

print(q.shape)

torch.Size([8, 16, 32])


In [10]:
import torch.nn.functional as F

wei = q @ k.transpose(-2, -1)
print(wei.shape) # shape(B, T, T)

tril = torch.tril(torch.ones(T, T))
print(tril)

wei = wei.masked_fill(tril == 0, float('-inf'))

# print(wei)

wei = F.softmax(wei, dim=-1)
print(wei)

x_attn = wei @ v

torch.Size([8, 16, 16])
tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1

# play MHA
- 思路1: 肉写,比较机械 key1,query1,value1, key2,query2,value2...
- 思路2: flatten一下，key_0to7。nn.Linear(embd_dim, header_dim * 8)
- 思路3: nn.ModuleList(),印象中Andrej是用的这个，但是具体怎么用还没想好
- 思路4: 之前问ChatGPT的实现，应该比较高效，是把多头的参数直接cat起来的

In [11]:
# 思路1
query1 = nn.Linear(embd_dim, header_dim)
key1 = nn.Linear(embd_dim, header_dim)
value1 = nn.Linear(embd_dim, header_dim)

query2 = nn.Linear(embd_dim, header_dim)
key2 = nn.Linear(embd_dim, header_dim)
value2 = nn.Linear(embd_dim, header_dim)

query3 = nn.Linear(embd_dim, header_dim)
key3 = nn.Linear(embd_dim, header_dim)
value3 = nn.Linear(embd_dim, header_dim)

query4 = nn.Linear(embd_dim, header_dim)
key4 = nn.Linear(embd_dim, header_dim)
value4 = nn.Linear(embd_dim, header_dim)

query5 = nn.Linear(embd_dim, header_dim)
key5 = nn.Linear(embd_dim, header_dim)
value5 = nn.Linear(embd_dim, header_dim)

query6 = nn.Linear(embd_dim, header_dim)
key6 = nn.Linear(embd_dim, header_dim)
value6 = nn.Linear(embd_dim, header_dim)

query7 = nn.Linear(embd_dim, header_dim)
key7 = nn.Linear(embd_dim, header_dim)
value7 = nn.Linear(embd_dim, header_dim)

query8 = nn.Linear(embd_dim, header_dim)
key8 = nn.Linear(embd_dim, header_dim)
value8 = nn.Linear(embd_dim, header_dim)

q1 = query1(x)
k1 = key1(x)
v1 = value1(x)

q2 = query2(x)
k2 = key2(x)
v2 = value2(x)

q3 = query3(x)
k3 = key3(x)
v3 = value3(x)

q4 = query4(x)
k4 = key4(x)
v4 = value4(x)

q5 = query5(x)
k5 = key5(x)
v5 = value5(x)

q6 = query6(x)
k6 = key6(x)
v6 = value6(x)

q7 = query7(x)
k7 = key7(x)
v7 = value7(x)

q8 = query8(x)
k8 = key8(x)
v8 = value8(x)

tril = torch.tril(torch.ones(T, T))

wei1 = q1 @ k1.transpose(-2, -1)
wei1 = wei1.masked_fill(tril == 0, float('-inf'))
wei1 = F.softmax(wei1, dim=-1)
x_header_1 = wei1 @ v1

wei2 = q2 @ k2.transpose(-2, -1)
wei2 = wei2.masked_fill(tril == 0, float('-inf'))
wei2 = F.softmax(wei2, dim=-1)
x_header_2 = wei2 @ v2

wei3 = q3 @ k3.transpose(-2, -1)
wei3 = wei3.masked_fill(tril == 0, float('-inf'))
wei3 = F.softmax(wei3, dim=-1)
x_header_3 = wei3 @ v3

wei4 = q4 @ k4.transpose(-2, -1)
wei4 = wei4.masked_fill(tril == 0, float('-inf'))
wei4 = F.softmax(wei4, dim=-1)
x_header_4 = wei4 @ v4

wei5 = q5 @ k5.transpose(-2, -1)
wei5 = wei5.masked_fill(tril == 0, float('-inf'))
wei5 = F.softmax(wei5, dim=-1)
x_header_5 = wei5 @ v5

wei6 = q6 @ k6.transpose(-2, -1)
wei6 = wei6.masked_fill(tril == 0, float('-inf'))
wei6 = F.softmax(wei6, dim=-1)
x_header_6 = wei6 @ v6

wei7 = q7 @ k7.transpose(-2, -1)
wei7 = wei7.masked_fill(tril == 0, float('-inf'))
wei7 = F.softmax(wei7, dim=-1)
x_header_7 = wei7 @ v7

wei8 = q8 @ k8.transpose(-2, -1)
wei8 = wei2.masked_fill(tril == 0, float('-inf'))
wei8 = F.softmax(wei8, dim=-1)
x_header_8 = wei8 @ v8 # shape(B, T, header_dim)

# 最后怎么合成到embd_dim，继续过FFN?
# shape(B, T, header_dim)，可以不考虑前面B, T 然后想象8个attn concat起来，过一个FC
# 这个等价于8个attn单独过一个自己的FC，然后再elementwise的add
# 所以继续发扬暴力写法。。。

fc1 = nn.Linear(header_dim, embd_dim)
fc2 = nn.Linear(header_dim, embd_dim)
fc3 = nn.Linear(header_dim, embd_dim)
fc4 = nn.Linear(header_dim, embd_dim)
fc5 = nn.Linear(header_dim, embd_dim)
fc6 = nn.Linear(header_dim, embd_dim)
fc7 = nn.Linear(header_dim, embd_dim)
fc8 = nn.Linear(header_dim, embd_dim)

x_mht = fc1(x_header_1) + fc2(x_header_2) + fc3(x_header_3) + fc4(x_header_4) + fc5(x_header_5) + fc6(x_header_6) + fc7(x_header_7) + fc8(x_header_8)

print(x_mht.shape) # shape=(8, 16, 256)

torch.Size([8, 16, 256])


* ChatGPT提到当模型结构较为复杂，需要灵活地管理子模块而不是简单地按顺序执行时，nn.ModuleList 提供了更大的灵活性。

In [12]:
# 思路3，自己从头写

query_list = nn.ModuleList([nn.Linear(embd_dim, header_dim)] * 8)
key_list = nn.ModuleList([nn.Linear(embd_dim, header_dim)] * 8)
value_list = nn.ModuleList([nn.Linear(embd_dim, header_dim)] * 8)

q_list = [query(x) for query in query_list]
k_list = [key(x) for key in key_list]
v_list = [value(x) for value in value_list]

x_header_list = list()
for n in range(8):
    wei = q_list[n] @ k_list[n].transpose(-2, -1)
    wei = wei.masked_fill(tril == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)
    x_header = wei @ v[n]
    x_header_list.append(x_header)

fc_list = nn.ModuleList([nn.Linear(header_dim, embd_dim)] * 8)

x_mht = torch.zeros(8, 16, embd_dim)
for n in range(8):
    fc = fc_list[n]
    x_mht += fc(x_header_list[n])

print(x_mht.shape)

torch.Size([8, 16, 256])


* 上面这种写法，感觉并行度不够，多头都是串行的。看看标准答案吧，哈哈。
* 标准答案思路：Header和MultiHeader 2个类，然后ModuleList 进来，循环过完再cat+proj
* 标准答案其实也有串行的部分！另外标准答案里面还有个dropout，这块之前没注意到
* 另外还有一个，忘了除以根号d, 原因上节课有讲，是为了保证vars在1.0左右，不至于让softmax太peak变成了hardmax哈哈

In [13]:
# 思路3，扫了下标准答案后
class Header(nn.Module):
    def __init__(self):
        super().__init__()
        self.query = nn.Linear(embd_dim, header_dim)
        self.key = nn.Linear(embd_dim, header_dim)
        self.value = nn.Linear(embd_dim, header_dim)

    def forward(self, x):
        q = self.query(x)
        k = self.key(x)
        v = self.value(x)

        wei = q @ k.transpose(-2, -1)

        # 标准答案
        # wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)

        tril = torch.tril(torch.ones(T, T))
        wei = wei.masked_fill(tril == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)

        # 标准答案
        # wei = self.dropout(wei)

        x_attn = wei @ v

        return x_attn

class MultiHead(nn.ModuleList):
    def __init__(self, headers_num=8):
        super().__init__()

        self.headers = nn.ModuleList([Header()] * headers_num)
        self.proj = nn.Linear(header_dim * headers_num, embd_dim)
        # 标准答案
        # self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x_header_list = [h(x) for h in self.headers] # shape (n, B, T, header_dim)
        x_headers_flatten = torch.cat(x_header_list, dim=-1)
        x_mha = self.proj(x_headers_flatten)

        return x_mha


multihead = MultiHead()
x_mha = multihead(x)

print(x_mha.shape)

torch.Size([8, 16, 256])


使用在gpt-mao的代码，对比一下标准答案，有如下分析：
1. **参数名更标准：** 标准答案参数名叫c_attn，这个是为了和Transformers库保持一致方便load模型
2. **更高效：** 标准答案使用了nn.Linear(config.n_embd, 3 * config.n_embd)，一次搞定qkv。
3. **遵循原论文：** shape从(B, T, C) -> (B, T, n_head, C // n_head) 保证了 C = n_head * head_size。 Think: 为啥要保证这个？
4. **核心逻辑一致：**  (B, T, n_head, head_size) -> (B, n_head, T, head_size)。Think: 为什么这么处理还是没特别想透

In [14]:
# 思路4:

class GPTConfig:
    def __init__(self, config):
        self.B = config['B']
        self.T = config['T']
        self.C = config['C']

        self.vocab_size = config['vocab_size']

        self.header_size = config['header_size']
        self.headers_num = config['headers_num']

        self.blocks_num = config['blocks_num']
        self.ffn_size = config['ffn_size']

class MultiHeadV4(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config

        self.key = nn.Linear(config.C, config.header_size * config.headers_num)
        self.query = nn.Linear(config.C, config.header_size * config.headers_num)
        self.value = nn.Linear(config.C, config.header_size * config.headers_num)
        self.attn_fc = nn.Linear(config.header_size * config.headers_num, config.C)


    def split_headers(self, x, batch_size):
        """Split the last dimension into (num_heads, head_dim) and transpose the result."""
        x = x.view(batch_size, config.T, self.config.headers_num, self.config.header_size)

        return x.transpose(1, 2) # shape (B, headers_num, T, header_size) 这里需要thinking一下why

    def combine_headers(self, x, batch_size):
        """
            Combine the heads and restore the original last dimension.
            x.shape (B, headers_num, T, header_size)
        """
        x = x.transpose(1, 2).contiguous()
        # 这里view中的-1，表示自动计算。其实可以直接用T
        return x.view(batch_size, self.config.T, self.config.headers_num * self.config.header_size)

    def attn(self, x):
        """
        x.shape (B, T, C)
        """
        config = self.config
        B, T, C = x.shape

        # k, q, v shape (B, headers_num, T, header_size)
        k = self.split_headers(self.key(x), B)
        q = self.split_headers(self.query(x), B)
        v = self.split_headers(self.value(x), B)

        tril = torch.tril(torch.ones(config.T, config.T))
        wei = q @ k.transpose(-2, -1) # 这个时候用负数的兼容性就体现了
        wei = wei.masked_fill(tril == 0, float('-inf'))
        wei = wei / (config.header_size ** 0.5)
        wei = F.softmax(wei, dim=-1)
        # wei.shape (B, headers_num, T, T)

        out = wei @ v # out.shape (B, headers_num, T, header_size)
        out = self.combine_headers(out, B)  # out.shape(B, T, config.header_size * config.headers_num)
        x = self.attn_fc(out)

        return x


    def forward(self, x):
        x = self.attn(x)
        return x


config = GPTConfig({'B': B, 'T': T, 'C': embd_dim, 'vocab_size': vocab_size, 'header_size': header_dim, 'headers_num': headers_num, 'blocks_num': 4, 'ffn_size': 256})
mha = MultiHeadV4(config)

x_mha_v4 = mha(x)
print(x_mha_v4.shape)

torch.Size([8, 16, 256])
